In [1]:
#!/usr/bin/python3
# -*-coding:utf-8
'''
Created on Fri Dec 1 22:22:35 2017

@author: Ray

'''
import time
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import utils # written by author
from glob import glob
from datetime import datetime, timedelta
import multiprocessing as mp
import gc # for automatic releasing memory


# trainW-0

In [102]:
T = 0
folder = 'trainW-'+ str(T)
train = pd.read_csv('../../input/preprocessed_data/trainW-{0}.csv'.format(T))[['msno']]
user_logs = utils.read_multiple_csv('../../feature/{}/user_lgos_listening_freq'.format(folder),parse_dates = ['date'])
#user_logs.sort_values(by = ['msno','date'], inplace = True)

100%|██████████| 10/10 [00:00<00:00, 138.30it/s]


In [103]:
#==============================================================================
print('reduce memory')
#==============================================================================
utils.reduce_memory(user_logs)
utils.reduce_memory(train)

reduce memory


In [104]:
# basic step
df = pd.merge(train,user_logs, on = 'msno', how = 'left')
df.sort_values(by = ['msno','date'], inplace = True)

In [105]:
df = df.dropna()

In [115]:
def t_diff(x, time_diff):
    for i in range(1, time_diff):
        x['t-{}_date'.format(i)] = x['date'].shift(i)
    return x
def within_n_days(x, T, n = 7):
    # n = 7, 14, 30, 60, 90
    ##################################################
    # Filtering accroding to w
    ##################################################
    if T == 0:
        # w = 0:使用3月之前的資料當作history
        prediction_deadline = datetime.strptime('2017-03-01', '%Y-%m-%d')
    elif T == 1:
        # w = 1:使用2月之前的資料當作history
        prediction_deadline = datetime.strptime('2017-02-01', '%Y-%m-%d')
    elif T == 2:
        # w = 2:使用1月之前的資料當作history
        prediction_deadline = datetime.strptime('2017-01-01', '%Y-%m-%d')
    elif T == -1:
        # w = -1:使用4月之前的資料當作history
        prediction_deadline = datetime.strptime('2017-04-01', '%Y-%m-%d')
    # n = 7, 14, 21, 30
    check_points = prediction_deadline - timedelta(n) # 往prediction_deadline前n天內
    x = x[ (x.date >= check_points )]
    # 如果有些使用者在預測deadline前n天內沒有資料,到時候再concat, merge的時候會是nan就把它補-1
    return x


In [107]:
input_col = ['msno','date']
df = df[input_col].groupby('msno').apply(t_diff,2)
df['date_diff'] = [i.days for i in (df.date - df['t-1_date']) ] # 每一個trainW-0只做一次
df = df.tail(n = df.shape[0] -1 ) # 第一筆不要
gc.collect()

67

In [108]:
#==============================================================================
print('reduce memory')
#==============================================================================
utils.reduce_memory(df)
df.drop(['t-1_date'], axis = 1, inplace = True)# saving memory
df = df.tail(n = df.shape[0] -1 )

reduce memory


In [112]:
df.head()

,msno,date,date_diff
767182,Pz51LVoS9ENG1kNHQyrJ3gG8A163pyHi+gyvN2p+1nM=,2015-06-16,1.0
767195,Pz51LVoS9ENG1kNHQyrJ3gG8A163pyHi+gyvN2p+1nM=,2015-06-17,1.0
767219,Pz51LVoS9ENG1kNHQyrJ3gG8A163pyHi+gyvN2p+1nM=,2015-06-18,1.0
767231,Pz51LVoS9ENG1kNHQyrJ3gG8A163pyHi+gyvN2p+1nM=,2015-06-19,1.0
767243,Pz51LVoS9ENG1kNHQyrJ3gG8A163pyHi+gyvN2p+1nM=,2015-06-20,1.0


In [110]:
# core
tbl = df[df.date_diff == 1].groupby('msno').date_diff.size().to_frame()
tbl.columns = ['num_listen_music_in_a_row_count']
tbl['num_listen_music_in_a_row_ratio'] = tbl.num_listen_music_in_a_row_count / df.groupby('msno').date_diff.apply(len)
tbl.reset_index(inplace = True)
tbl

,msno,num_listen_music_in_a_row_count,num_listen_music_in_a_row_ratio
0,Pz51LVoS9ENG1kNHQyrJ3gG8A163pyHi+gyvN2p+1nM=,584,0.971714


In [128]:
	##################################################
	# n = 7
	##################################################
	df_ = df.groupby('msno').apply(within_n_days, T, n = 7).reset_index(drop = True)
	#core
	tbl = df_[df_.date_diff == 1].groupby('msno').date_diff.size().to_frame()
	tbl.columns = ['listen_music_in_a_row_count_during_t_7']
	tbl['listen_music_in_a_row_ratio_during_t_7'] = tbl.listen_music_in_a_row_count_during_t_7 / df_.groupby('msno').date_diff.apply(len)
	tbl.reset_index(inplace = True)


In [129]:
tbl

,msno,listen_music_in_a_row_count_during_t_7,listen_music_in_a_row_ratio_during_t_7
0,Pz51LVoS9ENG1kNHQyrJ3gG8A163pyHi+gyvN2p+1nM=,4,0.8


In [126]:
##################################################
# n = 14
##################################################
df_ = df.groupby('msno').apply(within_n_days, T, n = 14).reset_index(drop = True)
#core
tbl = df_[df_.date_diff == 1].groupby('msno').date_diff.size().to_frame()
tbl.columns = ['listen_music_in_a_row_count_during_t_14']
tbl['listen_music_in_a_row_ratio_during_t_14'] = tbl.listen_music_in_a_row_count_during_t_14 / df_.groupby('msno').date_diff.apply(len)
tbl.reset_index(inplace = True)

In [127]:
tbl

,msno,listen_music_in_a_row_count_during_t_14,listen_music_in_a_row_ratio_during_t_14
0,Pz51LVoS9ENG1kNHQyrJ3gG8A163pyHi+gyvN2p+1nM=,9,0.818182


# test

In [25]:
folder = 'test'
user_logs = utils.read_multiple_csv('../../feature/{}/user_lgos_listening_behavior'.format(folder)) 


100%|██████████| 10/10 [00:00<00:00, 394.34it/s]


In [26]:
user_logs[user_logs.msno == 'Pz51LVoS9ENG1kNHQyrJ3gG8A163pyHi+gyvN2p+1nM=']

,msno,date,num_25,num_50,num_75,num_985,num_100,num_unq,num_completed_songs,num_incompleted_songs,completed_songs_ratio,is_satisfied,num_repeated_songs
0,Pz51LVoS9ENG1kNHQyrJ3gG8A163pyHi+gyvN2p+1nM=,2015-07-01,6,1,1,0,14,22,14,8,0.636364,1,0.681818
1,Pz51LVoS9ENG1kNHQyrJ3gG8A163pyHi+gyvN2p+1nM=,2015-09-01,21,3,0,1,42,59,43,24,0.641791,1,0.728814
2,Pz51LVoS9ENG1kNHQyrJ3gG8A163pyHi+gyvN2p+1nM=,2015-10-23,18,0,4,3,72,92,75,22,0.773196,1,0.858696
3,Pz51LVoS9ENG1kNHQyrJ3gG8A163pyHi+gyvN2p+1nM=,2015-12-13,1,0,0,0,21,21,21,1,0.954545,1,1.000000
4,Pz51LVoS9ENG1kNHQyrJ3gG8A163pyHi+gyvN2p+1nM=,2016-02-01,10,3,0,1,38,48,39,13,0.750000,1,0.812500
5,Pz51LVoS9ENG1kNHQyrJ3gG8A163pyHi+gyvN2p+1nM=,2016-03-22,2,0,0,1,27,29,28,2,0.933333,1,0.965517
6,Pz51LVoS9ENG1kNHQyrJ3gG8A163pyHi+gyvN2p+1nM=,2016-05-12,12,4,5,6,109,104,115,21,0.845588,1,1.153846
7,Pz51LVoS9ENG1kNHQyrJ3gG8A163pyHi+gyvN2p+1nM=,2016-07-02,16,2,1,4,27,45,31,19,0.620000,1,0.711111
8,Pz51LVoS9ENG1kNHQyrJ3gG8A163pyHi+gyvN2p+1nM=,2016-08-21,0,0,0,0,1,1,1,0,1.000000,1,1.000000
9,Pz51LVoS9ENG1kNHQyrJ3gG8A163pyHi+gyvN2p+1nM=,2016-10-10,14,2,3,6,63,74,69,19,0.784091,1,0.972973


In [7]:
user_logs

,date,msno,num_100,num_25,num_50,num_75,num_985,num_unq,total_secs_percentage,listening_habit_zone,num_of_time_the_user_has_logged_in,num_of_time_the_user_has_logged_in_ratio
3790,2015-08-18,RU75VCMW4eB6wHm1OFHyhK5/kPRikRYEET1jaLFobiI=,-1,-1,-1,-1,-1,-1,0.237463,2,24221.978516,6055.494629
4238,2016-01-30,p+59mhC3bE5x1sejuoIKGcq84z8VzSBL51GoCMTgYF4=,-1,-1,-1,-1,-1,-1,0.202898,2,42537.332031,14179.110352
100,2016-12-02,8WtZy307X8qfOUtuNIPFAyckcBlbQvOHn19DMsYCkcs=,-1,-1,-1,-1,-1,-1,0.037327,4,10437.060547,5218.530273
5327,2016-09-04,RKYtu5T1O7nLzdKk9sj75LVI8BlGIw0Tzbev64L5eLM=,-1,-1,-1,-1,-1,-1,0.025866,4,24547.656250,3068.457031
5401,2017-02-27,0EquqhtrfzpwJjGLCuzBvEArnqBRBsp5UoIocCzA32I=,-1,-1,-1,-1,-1,-1,0.049611,3,15469.943359,3867.485840
119,2015-12-23,Jgys/AsreAeCWb8Lv4ruvf/9D75Prq3b2msq2mdGtH8=,-1,-1,-1,-1,-1,-1,0.011170,4,11374.484375,1895.747437
918,2016-03-29,3ixxoForQ+NIP99dFQvu5wU3Q2D/pXgF+U3+eMTvSBI=,-1,-1,-1,-1,-1,-1,0.038321,4,43741.640625,8748.328125
5746,2016-10-15,KhKyYtYxGsb0lrE8qtz+zOlj9gUnWEkZoKuPucoiEXY=,-1,-1,-1,-1,-1,-1,0.488007,2,168716.937500,21089.617188
5637,2016-04-23,foZhV2R6MzY54/kzTY5ACmaddm1al2ukkimVhjoVEg8=,-1,-1,-1,-1,-1,-1,0.035596,4,3075.507080,3075.507080
3297,2016-11-14,sC9vlNuGvWZxd1pp1sLZqbkEdS1cs7oPJYL83MgdDnA=,-1,-1,-1,-1,-1,-1,0.094881,3,145071.484375,14507.148438
